In [ ]:
import json
import pandas as pd
import seaborn as sns
import numpy as np
import os
os.chdir('../src/')
from data_treatment import update_dataframe, update_all
from make_figures import do_SR, do_SR_mesa, do_2dplot, do_all
from os import listdir
from os.path import isfile, join
%matplotlib inline

datadir = '../data/'
data_files = [f for f in listdir(datadir) if isfile(join(datadir, f))]

df = [pd.read_json("../data/"+data,lines=True) for data in data_files]
df_final = pd.concat(df)

In [ ]:
variables = [u'ABSTENCION', u'ACTAS', u'ACTAS ESCRUTADAS', u'ACTAS VALIDAS',
     u'ELECTORES ACTAS TRANSMITIDAS', u'ELECTORES INSCRITOS',
       u'VOTANTES ESCRUTADOS', u'VOTOS ESCRUTADOS', u'VOTOS NULOS',
       u'VOTOS VALIDOS', u'centro', u'estado', u'mesa',u'municipio', u'parroquia','Abstencion_%',
       u'PSUV_%',u'PSUV_N',u'MUD_%',u'MUD_N', u'rescaled_N',u'parroquia_residual',
       u'municipio_residual',u'centro_residual',u'estado_residual',
       u'parroquia_standardised_residual', u'municipio_standardised_residual',u'centro_standardised_residual',
          u'estado_standardised_residual',u'parroquia_standardised_residual_abstention',]


df_out = update_all(df_final)[variables]



EDO. AMAZONAS


data_treatment.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['Abstencion_%'] = df['ABSTENCION'] / df['ELECTORES INSCRITOS']
data_treatment.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['turnout'] = 1 - df['Abstencion_%']
data_treatment.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['PSUV_%

EDO. ANZOATEGUI
EDO. APURE
EDO. ARAGUA
EDO. BARINAS
EDO. BOLIVAR
EDO. CARABOBO
EDO. COJEDES
EDO. DELTA AMACURO
EDO. FALCON
EDO. GUARICO
EDO. LARA
EDO. MERIDA
EDO. MIRANDA
EDO. MONAGAS
EDO. NVA.ESPARTA
EDO. PORTUGUESA


In [ ]:
df_out.shape[0]

df_out.dropna(inplace=True)

df_out = df_out[df_out['PSUV_%']<1.001]

    


# Data:

Results from Merida, Vargas, Monagas and Delta Amacuro.

Data unit is the result in each 'mesa' of an electoral centre. 

In [ ]:
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde

sns.pairplot(df_out,vars=['Abstencion_%','PSUV_%']);

fig, ax = plt.subplots()
xy = np.vstack([df_out['Abstencion_%'], df_out['PSUV_%']])
z = gaussian_kde(xy)(xy)
ax.scatter(df_out['Abstencion_%'], df_out['PSUV_%'], c=z, s=50)
plt.xlabel('parroquia_standardised_residual')
plt.ylabel('parroquia_standardised_residual_abstention')
plt.show()

#c = plt.contour(df_out['parroquia_standardised_residual'], df_out['parroquia_standardised_residual_abstention'],z,colors='r',linewidths=4,linestyles='dotted')
#plt.show()

In [ ]:
do_2dplot(df_out,'Abstencion_%','PSUV_%')


# Metrics:

Main metric is the Standardised residuals (SR) or Z-score:


residual_centro = $(1 -\frac{\%Psuv_{mesa}}{\%Psuv_{centro}})$ 

residual_parroquia = $(1 -\frac{\%Psuv_{mesa}}{\%Psuv_{parroquia}})$

$\underline{Standardised Residual}$ = $\frac{residual}{\sigma_{residual}}$ for each level (centro, parroquia, municipio...)

Deviations in favour of PSUV are shown on negative values of SR.

# Standardised residual for each mesa with respect to its centro (only centros with more than 1 'mesa'):

Gaussian distribution, if an external source affects the result in a 'mesa', it will affect the whole 'centro electoral'.

No large relative effect observed (slight tails), this can be our control sample. 


In [ ]:
do_SR(df_out)

# Standardised residual for each mesa with respect to its 'parroquia':

Observing slighly larger tails on the negative side, where PSUV would show larger percentange of votes wrt the result in the parroquia.

All histograms are normalised by number of valid votes in the centre.


In [ ]:
do_SR(df_out)


In [ ]:
do_SR_mesa(df_out)


# RESULTS PER STATE:

Effect changes from state to state:
## MERIDA

Visible asymmetry on tails for the parroquia SR.

In [ ]:
do_all(df_out_Merida)
#do_SR(df_out_Merida)

In [ ]:
do_SR_mesa(df_out_Merida)

## VARGAS

In [ ]:
do_SR(df_out_Vargas)

In [ ]:
do_SR_mesa(df_out_Vargas)

## MONAGAS

In [ ]:
do_SR(df_out_MONAGAS)

In [ ]:
do_SR_mesa(df_out_MONAGAS)

## Delta Amacuro

In [ ]:
do_SR(df_deltaA)

In [ ]:
do_SR_mesa(df_deltaA)

## MIRANDA

In [ ]:
do_SR(df_MIRANDA)



In [ ]:
print wtf

In [ ]:
do_SR_mesa(df_MIRANDA)


## LARA

In [ ]:
df_out_Lara.dropna(axis=0,inplace=True)

do_SR(df_out_Lara)


In [ ]:
do_SR_mesa(df_out_Lara)

